In [1]:
import numpy as np

from math import pi

from qiskit.circuit import QuantumCircuit, Parameter

from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp




$ H(x) = - \sum_i v_i x_i + A(\sum_i w_i x_i - C)^2 $

$A$ is a penalty constant $>$ max($w_i$)

Map binary $x_i = (1 - Z_i) / 2$ into Pauli-Z operators

In [5]:
def create_knapsack_hamiltonian(weights, vals, cap, penalty):
    n = len(vals)

    Q = np.zeros((n, n))
    lin = np.zeros(n)

    # obj: -v_i x_i
    lin -= np.array(vals)
    
    for i in range(n):
        lin[i] += penalty * (weights[i] ** 2 - 2 * cap * weights[i])
        for j in range(i + 1, n):
            Q[i, j] += 2 * penalty * weights[i] * weights[j]

    # const = penalty * cap ** 2

    # map to hamiltonian / ising
    paulis = []
    coeffs = []

    for i in range(n):
        # linear term
        if abs(lin[i]) > 1e-9:
            coeffs.append(lin[i] / 2)
            paulis.append("I" * (n - i - 1) + "Z" + "I" * i)

    for i in range(n):
        for j in range(i + 1, n):
            if abs(Q[i, j]) > 1e-9:
                coeffs.append(Q[i, j] / 4)
                label = ["I"] * n
                label[n - i - 1] = "Z"
                label[n - j - 1] = "Z"
                paulis.append("".join(label))
    
    hamiltonian = SparsePauliOp.from_list(list(zip(paulis, coeffs)))
    return hamiltonian
    

In [6]:
vals = [5, 4, 6]
weights = [2, 3, 4]
cap = 5
pen = 10 # > max(weights)

# problem hamiltonian
hamiltonian = create_knapsack_hamiltonian(weights, vals, cap, pen)

print(hamiltonian)

SparsePauliOp(['IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI'],
              coeffs=[ -82.5+0.j, -107. +0.j, -123. +0.j,   30. +0.j,   40. +0.j,   60. +0.j])
